In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import plotly.express as px
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
INPUT_SIZE_1 = 3
OUTPUT_SIZE_1 = 1

# in:
#   weekday
#   today
#   product label
# out:
#   days in storage

labels = {
    0: 5030,
    1: 5409
}

class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        INTERNAL = 7
        self.l1 = nn.Linear(INPUT_SIZE_1, INTERNAL)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(INTERNAL, OUTPUT_SIZE_1)
    
    def forward(self, x):
        return self.l2(self.relu(self.l1(x)))

In [64]:
class Data1(torch.utils.data.Dataset):
    def __init__(self, filepath):
        data = pd.read_csv(filepath)
        out = pd.DataFrame()
        out['product_id'] = data['product_id']
        out['production_date'] = data['PRODUCTION DATE'].map(lambda x: pd.to_datetime(x).value // 10**12)
        m = out['production_date'].min()
        out['days_in_storage'] = data['days_in_storage']
        out['weekday'] = data['weekday']
        self.data = out
    
    def __len__(self):
        return self.data.count()

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        x = row.drop(labels=['days_in_storage']).values
        y = row.drop(labels=['production_date', 'weekday', 'product_id']).values
        return torch.tensor(x, dtype=torch.float), torch.tensor(y, dtype=torch.float)

data1 = Data1('./time.csv')
data1.data

C:\Users\ktnlvr\AppData\Local\Temp\ipykernel_18024\3228810960.py:6: UserWarning:

Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



,product_id,production_date,days_in_storage,weekday
0,5409,1694304,1.0,0.0
1,5409,1694304,1.0,0.0
2,5409,1694304,1.0,0.0
3,5409,1709164,1.0,3.0
4,5409,1709164,1.0,3.0
...,...,...,...,...
11749,5409,1710028,1.0,3.0
11750,5409,1710028,3.0,3.0
11751,5409,1710028,1.0,3.0
11752,5409,1710028,4.0,3.0


In [67]:
model1 = Net1()
criterion = nn.MSELoss()
optimizer = optim.SGD(model1.parameters(), lr=0.001)
epochs = 1500

costval = []

for i in range(epochs):
    for j, (x_train, y_train) in enumerate(data1):
        y_pred = model1(x_train)
        cost = criterion(y_pred, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    if i % 150 == 0:
        print(cost)
        costval.append(cost)

tensor(4.2018e+19, grad_fn=<MseLossBackward0>)


KeyboardInterrupt: 